# 3 Tile tract and patch lists

In this notebook we will generate the list of patches that are present on a given VISTA tile. We will do so such that the tile is completely covered. That is we aim to provide every patch that contains any region of the tile.

In [1]:
from astropy.table import Table, Column, vstack # Perhaps we should use LSST tables

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
VIDEO_IN = './data/video_images_overview_20200820.csv'
VHS_IN = './data/vhs_images_overview.csv'
video_ims = Table.read(VIDEO_IN)
vhs_ims = Table.read(VHS_IN)




In [3]:
len(vhs_ims),len(video_ims)

(568965, 137269)

In [4]:
def fileToType(filename):
    filetype = ''
    types = {
        'tile':'_tl.fit',
        'stack':'_st.fit',
    }
    for k,v in types.items():
        #print(k,v)
        if filename.endswith(v):
            filetype = k
       
    return filetype
video_ims['type'] = [fileToType(f) for f in video_ims['file']]
vhs_ims['type'] = [fileToType(f) for f in vhs_ims['file']]

In [5]:
video_ims = video_ims[video_ims['type']=='tile']
vhs_ims = vhs_ims[vhs_ims['type']=='tile']

In [6]:
len(vhs_ims),len(video_ims)

(21747, 2205)

In [7]:
#We are using a rings skymap
from lsst.geom import SpherePoint 
from lsst.geom import degrees
from lsst.skymap.ringsSkyMap import RingsSkyMap, RingsSkyMapConfig

#Following taken from dmu1/2_Survey_comparisons.ipynb test choice
coord = SpherePoint(35.429025*degrees,-4.90853*degrees)
config = RingsSkyMapConfig()
#These config options are chose to be the same as HSC:
#https://github.com/lsst/obs_subaru/blob/master/config/hsc/makeSkyMap.py
#and copied for obs_vista
#https://github.com/lsst-uk/obs_vista/blob/master/config/makeSkyMap.py
config.numRings = 120
config.projection = "TAN"
config.tractOverlap = 1.0/60 # Overlap between tracts (degrees)
config.pixelScale = 0.168
sm = RingsSkyMap(config)
sm.findTract(coord)

TractInfo(id=8524, ctrCoord=[0.8087076683756161, 0.5811684769959703, -0.09074749834931431])

In [8]:
## Test on know tile

In [9]:
#Tile from dmu4/dmu4_Example
EX_TILE = "20121122/v20121122_00088_st_tl.fit"
ex_row = video_ims[
    video_ims['file'] == "/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/"+EX_TILE
][0]
ex_row

file,ra,dec,ra_0_0,ra_0_y,ra_x_0,ra_x_y,dec_0_0,dec_0_y,dec_x_0,dec_x_y,filter,size,hash,type
str98,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,int64,str32,str5
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20121122/v20121122_00088_st_tl.fit,35.43265,-4.72313,36.1259211592099,36.13048720302681,34.91284867786413,34.914758535953574,-4.074792762244068,-5.561089113115699,-4.077214438128979,-5.563516534082509,Ks,216357120,95f818d1b99dcee749b276666480cdc6,tile


In [10]:
ex_row['ra_0_0']

36.1259211592099

In [11]:
patches = sm.findTractPatchList(
        [
            SpherePoint(ex_row['ra_0_0']*degrees,ex_row['dec_0_0']*degrees),
            SpherePoint(ex_row['ra_0_y']*degrees,ex_row['dec_0_y']*degrees),
            SpherePoint(ex_row['ra_x_0']*degrees,ex_row['dec_x_0']*degrees),
            SpherePoint(ex_row['ra_x_y']*degrees,ex_row['dec_x_y']*degrees)
        ]
    )
t = patches[0][0]

In [12]:
patches

[(TractInfo(id=8766, ctrCoord=[0.8103482053641914, 0.5823474300594684, -0.06486336998774297]),
  (PatchInfo(index=(2, 0), innerBBox=(minimum=(8000, 0), maximum=(11999, 3999)), outerBBox=(minimum=(7900, 0), maximum=(12099, 4099))),
   PatchInfo(index=(2, 1), innerBBox=(minimum=(8000, 4000), maximum=(11999, 7999)), outerBBox=(minimum=(7900, 3900), maximum=(12099, 8099))),
   PatchInfo(index=(2, 2), innerBBox=(minimum=(8000, 8000), maximum=(11999, 11999)), outerBBox=(minimum=(7900, 7900), maximum=(12099, 12099))),
   PatchInfo(index=(3, 0), innerBBox=(minimum=(12000, 0), maximum=(15999, 3999)), outerBBox=(minimum=(11900, 0), maximum=(16099, 4099))),
   PatchInfo(index=(3, 1), innerBBox=(minimum=(12000, 4000), maximum=(15999, 7999)), outerBBox=(minimum=(11900, 3900), maximum=(16099, 8099))),
   PatchInfo(index=(3, 2), innerBBox=(minimum=(12000, 8000), maximum=(15999, 11999)), outerBBox=(minimum=(11900, 7900), maximum=(16099, 12099))),
   PatchInfo(index=(4, 0), innerBBox=(minimum=(16000, 0

In [13]:
import json
tp_dict ={}
for tract in patches:
    #print(tract[0].getId())
    #tp_dict[tract[0].getId()] = [[t.getIndex()[0],t.getIndex()[1]] for t in tract[1]]
    tp_dict[int(tract[0].getId())] = [t.getIndex() for t in tract[1]]

j = json.dumps(tp_dict, separators=(',', ':')) 
j

'{"8766":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],"8524":[[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],"8765":[[0,0],[0,1],[0,2]],"8523":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8]]}'

In [14]:
def corners_to_patch_list(ex_row):
    """Take a ra dec limited region and return a list of all patches contained witing it
    
    Inputs
    =======
    ex_row    astropy.table.row
              Row of image overview table containing corner columns
    
    Returns
    =======
    j        str
             String which can be loaded by json to create a dictionary
             the tracts indices are strings as required by json
    """
    try:
        patches = sm.findTractPatchList(
            [
                SpherePoint(ex_row['ra_0_0']*degrees,ex_row['dec_0_0']*degrees),
                SpherePoint(ex_row['ra_0_y']*degrees,ex_row['dec_0_y']*degrees),
                SpherePoint(ex_row['ra_x_0']*degrees,ex_row['dec_x_0']*degrees),
                SpherePoint(ex_row['ra_x_y']*degrees,ex_row['dec_x_y']*degrees)
            ]
        )
    
        tp_dict ={}
        for tract in patches:
            tp_dict[int(tract[0].getId())] = [t.getIndex() for t in tract[1]]
        j = json.dumps(tp_dict, separators=(',', ':')) 
    except:
        print(ex_row['file']+" failed")
        j=''
    return j
corners_to_patch_list(video_ims[0])

'{"8766":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],"8524":[[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],"8765":[[0,0],[0,1],[0,2]],"8523":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8]]}'

In [15]:
video_ims['tract_patch_json'] = [corners_to_patch_list(row) for row in video_ims]
vhs_ims['tract_patch_json'] = [corners_to_patch_list(row) for row in vhs_ims]
#corners_to_patch_list(video_ims[0])

/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20140924/v20140924_00570_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20140924/v20140924_00606_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20140924/v20140924_00594_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20140924/v20140924_00582_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20141029/v20141029_00424_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20141029/v20141029_00412_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20141029/v20141029_00460_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20141029/v20141029_00472_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20141029/v20141029_00436_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20141029/v20141029_00448_st_tl.fit failed
/home/ir-s

/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110109/v20110109_00169_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110109/v20110109_00380_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110109/v20110109_00241_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110109/v20110109_00356_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110109/v20110109_00121_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110109/v20110109_00097_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110109/v20110109_00229_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110109/v20110109_00368_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110109/v20110109_00205_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110109/v20110109_00193_st_tl.fit failed
/home/ir-s

/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100612/v20100612_00340_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100612/v20100612_00700_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100612/v20100612_00676_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100612/v20100612_00052_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100612/v20100612_00388_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100612/v20100612_00280_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100612/v20100612_00508_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100612/v20100612_00304_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100612/v20100612_00760_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100612/v20100612_00748_st_tl.fit failed
/home/ir-s

/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100213/v20100213_00293_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100213/v20100213_00619_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100213/v20100213_00739_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100213/v20100213_00317_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100213/v20100213_00835_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100213/v20100213_00515_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100213/v20100213_00245_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100213/v20100213_00365_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100213/v20100213_00281_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20100213/v20100213_00643_st_tl.fit failed
/home/ir-s

/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110612/v20110612_00522_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110612/v20110612_00396_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110612/v20110612_00312_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20130121/v20130121_00252_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20130121/v20130121_00516_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20130121/v20130121_00432_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20130121/v20130121_00054_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20130121/v20130121_00528_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20130121/v20130121_00288_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20130121/v20130121_00384_st_tl.fit failed
/home/ir-s

...

/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110626/v20110626_00520_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110626/v20110626_00460_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110626/v20110626_00700_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110626/v20110626_00220_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110626/v20110626_00352_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110626/v20110626_00448_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110626/v20110626_00832_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110626/v20110626_00868_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110626/v20110626_00688_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110626/v20110626_00388_st_tl.fit failed
/home/ir-s

/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120518/v20120518_00653_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120518/v20120518_00461_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120518/v20120518_00259_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120518/v20120518_00389_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120518/v20120518_00353_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120518/v20120518_00689_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120518/v20120518_00473_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120518/v20120518_00569_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120518/v20120518_00377_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120518/v20120518_00581_st_tl.fit failed
/home/ir-s

/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110930/v20110930_00385_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110930/v20110930_00517_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110117/v20110117_00176_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110117/v20110117_00152_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110117/v20110117_00164_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20101113/v20101113_00151_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20101113/v20101113_00175_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20101113/v20101113_00211_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20101113/v20101113_00163_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20101113/v20101113_00254_st_tl.fit failed
/home/ir-s

/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120531/v20120531_00827_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120531/v20120531_00695_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120531/v20120531_00575_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120531/v20120531_00779_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120531/v20120531_00647_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120531/v20120531_00635_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120531/v20120531_00479_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120531/v20120531_00743_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120531/v20120531_00864_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20120531/v20120531_00491_st_tl.fit failed
/home/ir-s

/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110224/v20110224_00001_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110224/v20110224_00013_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110224/v20110224_00109_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110224/v20110224_00025_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110224/v20110224_00085_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110224/v20110224_00097_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110224/v20110224_00037_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20110224/v20110224_00049_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20130226/v20130226_00222_st_tl.fit failed
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20130226/v20130226_00186_st_tl.fit failed
/home/ir-s

In [16]:
video_ims[:5]

file,ra,dec,ra_0_0,ra_0_y,ra_x_0,ra_x_y,dec_0_0,dec_0_y,dec_x_0,dec_x_y,filter,size,hash,type,tract_patch_json
str98,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,int64,str32,str5,str631
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20140924/v20140924_00444_st_tl.fit,35.429542,-4.72215,36.12993723683939,36.13433423332165,34.912894878395214,34.91462725348546,-4.072782101046067,-5.559458172880838,-4.07547203760631,-5.562153562881393,Y,148561920,ccef567359a9d3f1cef197b5b5dad958,tile,"{""8766"":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],""8524"":[[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""8765"":[[0,0],[0,1],[0,2]],""8523"":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20140924/v20140924_00492_st_tl.fit,52.63885,-27.48791,53.376808661281494,51.6905687690388,53.363677671568034,51.69601464898924,-28.176115497300387,-28.180884566723634,-26.96066324597599,-26.965386169492707,J,174971520,68fe6e1c16640525f9137e139ae269a6,tile,"{""4849"":[[6,8],[7,8],[8,8]],""5063"":[[3,0],[3,1],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[4,0],[4,1],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[5,0],[5,1],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[6,0],[6,1],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[7,0],[7,1],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[8,0],[8,1],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7]],""4848"":[[0,8],[1,8],[2,8],[3,8],[4,8],[5,8],[6,8]],""5062"":[[0,0],[0,1],[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[1,0],[1,1],[1,2],[1,3],[1,4],[1,5],[1,6],[1,7],[2,0],[2,1],[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[3,0],[3,1],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20140924/v20140924_00396_st_tl.fit,35.428004,-4.7219,36.130230295324694,36.134884600391395,34.913103896675054,34.91508881101303,-4.071964313054827,-5.561119342013037,-4.074296412803666,-5.5634573489486225,Y,146859840,2b3c420a7d15853f89f53c893ef9526e,tile,"{""8766"":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],""8524"":[[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]],""8765"":[[0,0],[0,1],[0,2]],""8523"":[[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[0,8]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20141029/v20141029_00364_st_tl.fit,52.637771,-27.48048,53.37745012628302,51.68913871643598,53.36462194039669,51.69488106361268,-28.174285005665155,-28.178915893328604,-26.960698293435776,-26.965286581454887,Y,152732160,77a38c51b547fca022c539ea718a47bf,tile,"{""4849"":[[6,8],[7,8],[8,8]],""5063"":[[3,0],[3,1],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7],[4,0],[4,1],[4,2],[4,3],[4,4],[4,5],[4,6],[4,7],[5,0],[5,1],[5,2],[5,3],[5,4],[5,5],[5,6],[5,7],[6,0],[6,1],[6,2],[6,3],[6,4],[6,5],[6,6],[6,7],[7,0],[7,1],[7,2],[7,3],[7,4],[7,5],[7,6],[7,7],[8,0],[8,1],[8,2],[8,3],[8,4],[8,5],[8,6],[8,7]],""4848"":[[0,8],[1,8],[2,8],[3,8],[4,8],[5,8],[6,8]],""5062"":[[0,0],[0,1],[0,2],[0,3],[0,4],[0,5],[0,6],[0,7],[1,0],[1,1],[1,2],[1,3],[1,4],[1,5],[1,6],[1,7],[2,0],[2,1],[2,2],[2,3],[2,4],[2,5],[2,6],[2,7],[3,0],[3,1],[3,2],[3,3],[3,4],[3,5],[3,6],[3,7]]}"
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VIDEO/20141029/v20141029_00316_st_tl.fit,35.437767,-4.72747,36.13052006717623,36.13523423715193,34.912898410633574,34.914941744138545,-4.072033411697982,-5.561286819567744,-4.074791337249168,-5.564050760740607,Y,154330560,6a8a3cfacd2

In [20]:
vhs_ims[-5:]

file,ra,dec,ra_0_0,ra_0_y,ra_x_0,ra_x_y,dec_0_0,dec_0_y,dec_x_0,dec_x_y,filter,size,hash,type,tract_patch_json
str96,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,int64,str32,str5,str1
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20140217/v20140217_00257_st_tl.fit,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,--,0,--,tile,
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20140217/v20140217_00281_st_tl.fit,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,--,0,--,tile,
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20140217/v20140217_00269_st_tl.fit,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,--,0,--,tile,
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20140217/v20140217_00245_st_tl.fit,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,--,0,--,tile,
/home/ir-shir1/rds/rds-iris-ip005/data/private/VISTA/VHS/20140217/v20140217_00293_st_tl.fit,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,--,0,--,tile,


In [17]:
video_ims[6]['tract_patch_json']

'{"3053":[[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]],"3054":[[2,0],[2,1],[2,2],[3,0],[3,1],[3,2],[4,0],[4,1],[4,2],[5,0],[5,1],[5,2],[6,0],[6,1],[6,2],[7,0],[7,1],[7,2],[8,0],[8,1],[8,2]],"2876":[[0,3],[0,4],[0,5],[0,6],[0,7],[0,8],[1,3],[1,4],[1,5],[1,6],[1,7],[1,8],[2,3],[2,4],[2,5],[2,6],[2,7],[2,8],[3,3],[3,4],[3,5],[3,6],[3,7],[3,8]],"2877":[[3,3],[3,4],[3,5],[3,6],[3,7],[3,8],[4,3],[4,4],[4,5],[4,6],[4,7],[4,8],[5,3],[5,4],[5,5],[5,6],[5,7],[5,8],[6,3],[6,4],[6,5],[6,6],[6,7],[6,8],[7,3],[7,4],[7,5],[7,6],[7,7],[7,8],[8,3],[8,4],[8,5],[8,6],[8,7],[8,8]]}'

In [21]:
video_ims.write(VIDEO_IN.replace('images','tiles_tracts_patches'),overwrite=True)
vhs_ims.write(VHS_IN.replace('images','tiles_tracts_patches'),overwrite=True)